In [1]:
import gc

import numpy as np

"""
ToyNetwork 재분석
    화성~서울 네트워크
    집계시간 : 5분
    분석시간 1800~13200
    유입 램프 1개
"""
import pandas as pd

import os

# FIX 값 모음
###################################################################################################################

path = r"C:\VISSIM_Workspace\화성~서울\기초분석\2차 분석\2. 화성~서울(유입램프 1개)"

start_interval = 1800
end_interval = 13200

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1
}

vehicle_types = [100, 300, 630, 640, 650]

# 램프 간섭 영향률
###################################################################################################################
# 램프 전 본선 검지기(램프 간섭 영향률)
before_ramp = [118, 312]

# 램프 후 본선 검지기(램프 간섭 영향률)
after_ramp = [121,317]

# 유입램프 검지기(램프 간섭 영향률)
input_ramp = [902, 906]

# 유출램프 검지기(램프 간섭 영향률)
output_ramp = [901, 903, 905, 907, 909]

# 진출 정상성(진입)(진출 원활률)
enter_line = [23, 121, 190, 317, 406]

# 유입램프 바로 뒤 본선 검지기(진출 원활률)(램프 간섭 영향률)
input_main_ramp = [121, 317]

# 유출램프 바로 앞 본선 검지기(진출 원활률)(램프 간섭 영향률)
output_main_ramp = [73, 126, 220, 344, 414]
###################################################################################################################

# 함수 모음
###################################################################################################################

# 속도 변화율
def speed_mean(original_df):
    copy_df = original_df.copy()

    # 램프 검지기 제외
    copy_df = copy_df[~copy_df["New_Measurement"].between(900, 910)]


    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    speed_mean_df["delta_V"] = speed_mean_df["delta_V"].fillna(0)
    return speed_mean_df

# 밀도 변화율
def density_mean(speed_df):
    copy_df = speed_df.copy()
    density_mean_df = copy_df.assign(K = copy_df["V_count"] * 12 / copy_df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    density_mean_df["delta_K"] = density_mean_df["delta_K"].fillna(0)

    return density_mean_df

# 중차량 혼입률
def heavy_rate(original_df):
    copy_df = original_df.copy()

    heavy_df = (
        copy_df[copy_df["Vehicle type"].isin([630, 640, 650])]
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="heavy_count")
    )
    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="total_count")
    )
    heavy_rate_df = pd.merge(
        heavy_df,
        total_df,
        on=["TimeGroup", "New_Measurement"],
        how="left"
    )
    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]
    heavy_rate_df["rate"] = heavy_rate_df["rate"].fillna(0)
    return heavy_rate_df


# 동적 포화도
def entry_saturation(original_df):
    copy_df = original_df.copy()

    # 실측용량 C(2차로 4400)
    max_capacity = 4400
    entry_saturation_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="entry_volume")  # 차량 수를 entry_volume이라는 컬럼명으로
    )

    # 단위가 대/시 이기 때문에 현재 5분집계 * 12
    entry_saturation_df["Phi_진입"] = entry_saturation_df["entry_volume"] * 12 / max_capacity
    return entry_saturation_df

# 램프 간섭 영향률
def rfr_rate(original_df):
    copy_df = original_df.copy()
    copy_df["TimeGroup"] = copy_df["TimeGroup"].astype(str)
    main_results=[]

    for i, (before, after) in enumerate(zip(before_ramp, after_ramp)):
        q_before = (copy_df[copy_df["New_Measurement"] == before] # 118, 312
                    .groupby("TimeGroup")
                    .size()
                    .reset_index(name="q_before"))

        q_after = (copy_df[copy_df["New_Measurement"] == after] # 121, 317
                   .groupby("TimeGroup")
                   .size()
                   .reset_index(name="q_after"))

        merged = q_before.merge(q_after, on="TimeGroup", how="outer").fillna(0)
        merged["Qm"] = (merged["q_before"] + merged["q_after"]) / 2
        main_results.append(merged)


    ramp_results = []
    for input_, output_ in zip(input_ramp, output_ramp):
        """
        q_out = (copy_df[copy_df["New_Measurement"] == output_] # 901, 903, 905, 907, 909
                 .groupby("TimeGroup").size().reset_index(name="q_out"))
        print(type(q_out))
        display("q_out : ", q_out)

        ramp_results.append(q_out)
        """
        q_in = (copy_df[copy_df["New_Measurement"] == input_] # 902, 906
                .groupby("TimeGroup").size().reset_index(name="q_in"))

        ramp_results.append(q_in)

    input_queue = input_main_ramp.copy()
    output_queue = output_main_ramp.copy()
    rfr_list = []

    for i in range(min(len(main_results), len(ramp_results))):
        main_df = main_results[i]
        ramp_df = ramp_results[i]

        rfr_df = pd.merge(main_df, ramp_df, on="TimeGroup", how="outer").fillna(0)

        # 기본값 초기화
        rfr_df["IR_in"] = 0
        rfr_df["IR_out"] = 0

        # q_in 있을 때 (유입)
        if "q_in" in rfr_df.columns:
            rfr_df["IR_in"] = rfr_df["q_in"] / rfr_df["Qm"]
            if input_queue:  # 남은 게 있으면 하나 꺼냄
                current_input = input_queue.pop(0)
                rfr_df["New_Measurement"] = current_input

        # q_out 있을 때 (유출)
        if "q_out" in rfr_df.columns:
            rfr_df["IR_out"] = rfr_df["q_out"] / rfr_df["Qm"]
            if output_queue:
                current_output = output_queue.pop(0)
                rfr_df["New_Measurement"] = current_output

        # 유출램프 없을 때
        rfr_df["IR_out"] = 0

        rfr_df["RFR"] = rfr_df["IR_in"] + rfr_df["IR_out"]

        rfr_list.append(rfr_df)

    final_rfr_df = pd.concat(rfr_list, ignore_index=True)
    # -----------------------------
    # 특정 검지기에만 RFR 반영
    # -----------------------------
    #target_measurements = input_main_ramp + output_main_ramp
    target_measurements = input_main_ramp
    all_measurements = copy_df["New_Measurement"].unique()

    expanded_df_list = []

    base_rfr_df = final_rfr_df.copy()

    for m in all_measurements:
        if m in target_measurements:
            temp = base_rfr_df[base_rfr_df["New_Measurement"] == m].copy()
        else:
            temp = base_rfr_df[["TimeGroup"]].drop_duplicates().copy()
            temp["New_Measurement"] = m
            temp["RFR"] = 0

        expanded_df_list.append(temp)

    final_rfr_df = pd.concat(expanded_df_list, ignore_index=True)
    final_rfr_df = final_rfr_df.sort_values(by=["TimeGroup", "New_Measurement"]).reset_index(drop=True)
    final_rfr_df = final_rfr_df[["TimeGroup", "New_Measurement", "RFR"]]
    final_rfr_df["RFR"] = final_rfr_df["RFR"].fillna(0)
    return final_rfr_df


# 진출 원활율- output_main_ramp
def output_normality(original_df):

    copy_df = original_df.copy()

    normality_list = []

    # 여러 진입/진출 쌍 처리
    for enter, exit_ramp, exit_main in zip(enter_line, output_ramp, output_main_ramp):

        entry_df = copy_df[copy_df["New_Measurement"] == enter][["New_Measurement", "VehNo", "t(Entry)"]]

        exit_df  = copy_df[copy_df["New_Measurement"] == exit_ramp][["New_Measurement", "VehNo", "t(Entry)"]]

        # 차량 번호별 최소 통과시각
        entry_first = (
            entry_df.groupby("VehNo")["t(Entry)"].min()
            .reset_index()
            .rename(columns={"t(Entry)": "t_entry"})
        )

        exit_first = (
            exit_df.groupby("VehNo")["t(Entry)"].min()
            .reset_index()
            .rename(columns={"t(Entry)": "t_exit"})
        )

        # 진입-진출 매칭 후 지연시간 계산
        merged = pd.merge(entry_first, exit_first, on="VehNo", how="inner")
        merged["delay_sec"] = merged["t_exit"] - merged["t_entry"]
        merged = merged[merged["delay_sec"] >= 0]  # 음수 제거

        # 중간값 기반 시간지연 bin 계산
        if len(merged) and np.isfinite(np.nanmedian(merged["delay_sec"])):
            lag_bins = int(round(np.nanmedian(merged["delay_sec"]) / 300))
        else:
            lag_bins = 0

        # 진입/진출 카운트 집계
        entry_count = (
            copy_df[copy_df["New_Measurement"] == enter]
            .groupby("TimeGroup")
            .size()
            .reset_index(name="Q_in")
        )

        exit_count = (
            copy_df[copy_df["New_Measurement"] == exit_ramp]
            .groupby("TimeGroup")
            .size()
            .reset_index(name="Q_out")
        )

        # 병합 후 지연만큼 shift
        merged_counts = pd.merge(entry_count, exit_count, on="TimeGroup", how="left")


        merged_counts["Q_out_shift"] = merged_counts["Q_out"].shift(-lag_bins)


        # F(outrate)
        merged_counts["F(outrate)"] = (merged_counts["Q_out_shift"] / merged_counts["Q_in"]).fillna(0)
        merged_counts["New_Measurement"] = exit_main  # 진출 지점에 부여

        normality_list.append(merged_counts)


    # 모든 진출 램프 결과 병합
    final_df = pd.concat(normality_list, ignore_index=True)


    # 전체 검지기 확장
    all_measurements = copy_df["New_Measurement"].unique()
    expanded_list = []

    for m in all_measurements:
        if m in output_main_ramp:
            temp = final_df[final_df["New_Measurement"] == m].copy()
        else:
            temp = final_df[["TimeGroup"]].drop_duplicates().copy()
            temp["New_Measurement"] = m
            temp["F(outrate)"] = 0
        expanded_list.append(temp)

    final_df = pd.concat(expanded_list, ignore_index=True)
    final_df = final_df.sort_values(by=["TimeGroup", "New_Measurement"]).reset_index(drop=True)

    final_df["F(outrate)"] = 0
    return final_df


def calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):

    # TimeGroup 기준으로  Merge
    merged_df = (
        speed_df[["TimeGroup", "New_Measurement", "delta_V"]]
        .merge(density_df[["TimeGroup", "New_Measurement", "delta_K"]], on=["TimeGroup", "New_Measurement"])
        .merge(heavy_df[["TimeGroup", "New_Measurement", "rate"]], on=["TimeGroup", "New_Measurement"])
        .merge(entry_saturation_df[["TimeGroup", "New_Measurement", "Phi_진입"]], on=["TimeGroup", "New_Measurement"])
        .merge(rfr_df[["TimeGroup", "New_Measurement", "RFR"]], on=["TimeGroup", "New_Measurement"])
        .merge(normality_df[["TimeGroup", "New_Measurement", "F(outrate)"]], on=["TimeGroup", "New_Measurement"])
    )

    merged_df["STVM"] = (
        weights["w1"] * merged_df["delta_V"] +
        weights["w2"] * merged_df["delta_K"] +
        weights["w3"] * merged_df["rate"] +
        weights["w4"] * merged_df["Phi_진입"] +
        weights["w5"] * merged_df["RFR"] +
        weights["w6"] * merged_df["F(outrate)"]
    )
    merged_df.replace([np.inf, -np.inf], 0, inplace=True)
    merged_df.fillna(0, inplace=True)
    merged_df = modify_frame(merged_df)
    return merged_df

def calc_z(df):
    if df.empty:
        return df
    mean_stvm = df["STVM"].mean(skipna=True)
    std_stvm = df["STVM"].std(skipna=True)
    df["Z-Score"] = (df["STVM"] - mean_stvm) / std_stvm
    z_max = df["Z-Score"].max()
    z_min = df["Z-Score"].min()
    df["환산점수"] = df["Z-Score"].apply(lambda z: z_to_score(z, z_min, z_max))
    return df

def calculate_z_score(stvm_df):
    copy_df = stvm_df.copy()

    # 구간 나누기
    group1 = copy_df[copy_df["New_Measurement"].between(1, 265)].copy()
    group2 = copy_df[copy_df["New_Measurement"].between(266, 530)].copy()

    group1 = calc_z(group1)
    group2 = calc_z(group2)

    # 두 구간 합치기
    merged = pd.concat([group1, group2], ignore_index=True)
    merged = merged.sort_values(by=["TimeGroup", "New_Measurement"])
    #save_to_excel(merged, folder_path, "환산점수 원시데이터", i)

    stvm_df = pd.pivot(merged, index="TimeGroup", columns= "New_Measurement", values="환산점수")

    return stvm_df

def modify_frame(df):
    modify_df = df.copy()
    modify_df["StartTime"] = modify_df["TimeGroup"].str.split("~").str[0].astype(int)
    modify_df = modify_df[(modify_df["StartTime"] >=start_interval) &(modify_df["StartTime"] < end_interval)]
    modify_df = modify_df[~modify_df["New_Measurement"].isin([266, 531, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910])]
    return modify_df


def variable_timegroup_avg(stvm_df):
    copy_df = stvm_df.copy()
    variable_time_df = copy_df.groupby("TimeGroup")[["delta_V", "delta_K", "rate", "Phi_진입", "RFR", "F(outrate)"]].mean()
    return variable_time_df

def variable_total_avg(variable_df):
    variable_total_df = pd.DataFrame([variable_df.mean(numeric_only=True)])
    return variable_total_df

def speed_density_avg(density_df):
    copy_df = density_df.copy()
    avg_df = modify_frame(copy_df)
    avg_df = pd.DataFrame([avg_df.mean(numeric_only=True)])
    avg_df = avg_df[["V_mean", "K"]]
    return avg_df

def pivot_table(df, value, preprocess=None):
    copy_df = df.copy()
    if preprocess :
        copy_df = preprocess(copy_df)
    copy_df = copy_df.pivot(index="TimeGroup", columns="New_Measurement", values=value)

    copy_df = (
        copy_df
        .assign(_t=lambda x: x.index.astype(str).str.split("~").str[0].astype(int))
        .sort_values("_t")
        .drop(columns="_t")
    )
    copy_df = copy_df.fillna(0)
    return copy_df

def stvm_color(val):
    if pd.isna(val):
        return ""
    elif val <= 0:
        return "background-color: #00B050" # 초록
    else:
        return "background-color: #FFC000"  # 주황색


def weighted_avg_speed(original_df):
    copy_df = original_df.copy()
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement", "Vehicle type"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["std_group"] = speed_mean_df.groupby(["TimeGroup", "New_Measurement"])["V_mean"].transform(lambda s: s.std(ddof=0))
    speed_mean_df["cv"] = speed_mean_df["std_group"] / speed_mean_df["V_mean"]
    speed_mean_df["w"] = 1 / speed_mean_df["cv"]
    speed_mean_df["w*v"] = speed_mean_df["w"] * speed_mean_df["V_mean"]

    weighted_result = (
        speed_mean_df.groupby(["TimeGroup","New_Measurement"])
          .apply(lambda g: g["w*v"].sum() / g["w"].sum())
          .reset_index(name="Weighted_Avg_Speed")
    )

    return weighted_result

def save_to_excel(excel_df, folder_path, file_name, i, color=False):
        excel_folder_path = os.path.join(folder_path, file_name)
        os.makedirs(excel_folder_path, exist_ok=True)
        excel_file_name = f"{file_name}_{i+1}.xlsx"
        excel_file_path = os.path.join(excel_folder_path, excel_file_name)

        if color:
            styled = excel_df.style.applymap(stvm_color)
            styled.to_excel(excel_file_path, engine="openpyxl")
        else:
            excel_df.to_excel(excel_file_path, index=True)

        print(f"{excel_file_name} 생성 완료")

def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * ((z - 1.645) / (z_max - 1.645))) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * ((z - 1.282) / (1.645 - 1.282))) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * ((z - 1.038) / (1.282 - 1.038))) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * ((z - 0.842) / (1.038 - 0.842))) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * ((z - 0.676) / (0.842 - 0.676))) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * ((z - 0.526) / (0.676 - 0.526))) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * ((z - 0.387) / (0.526 - 0.387))) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * ((z - 0.255) / (0.387 - 0.255))) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * ((z + 0.255) / (0.255 + 0.255))) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * ((z + 0.387) / (-0.255 + 0.387))) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * ((z + 0.526) / (-0.387 + 0.526))) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * ((z + 0.676) / (-0.676 + 0.842))) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * ((z + 0.842) / (-0.676 + 0.842))) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * ((z + 1.038) / (-0.842 + 1.038))) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * ((z + 1.282) / (-1.038 + 1.282))) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * ((z + 1.645) / (-1.282 + 1.645))) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * ((z + z_min) / (-1.645 + z_min))) * 0.5)
    else:
        return np.nan
###################################################################################################################

folder_path = path
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]

#mer_list = ["화성~서울(유고)_260108_003.mer"]

stvm_df_list = []

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()

for i in range(len(mer_list)):
    mer_file = mer_list[i]
    print("작업파일 : ", mer_file)
    with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)

                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, 300)
                labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)

                # 가감속 차로 부분 검지기 제거
                original_df = original_df[~original_df["Measurem."].between(30000, 39999)]

                # 속도변화율
                speed_df = speed_mean(original_df)
                #save_to_excel(speed_df, folder_path, "속도", i)

                # 밀도변화율
                density_df = density_mean(speed_df)
                #save_to_excel(density_df, folder_path, "밀도변화량", i)

                # 중차량 혼입률
                heavy_df = heavy_rate(original_df)

                # 동적 포화도
                entry_saturation_df = entry_saturation(original_df)

                # 램프 간섭 영향률
                rfr_df = rfr_rate(original_df)

                # 진출 원활율
                normality_df = output_normality(original_df)

                # STVM 계산
                stvm_df = calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)
                #display("stvm_df : ", stvm_df)
                save_to_excel(stvm_df, folder_path, "STVM", i)

                # Z-Score 계산
                #z_score_df = calculate_z_score(stvm_df)
                #save_to_excel(z_score_df, folder_path, "환산점수", i)

                # STVM 피봇
                #stvm_pivot_df = pivot_table(stvm_df, "STVM", preprocess=modify_frame)
                #display("stvm_pivot_df : ", stvm_pivot_df)
                #save_to_excel(stvm_pivot_df, folder_path, "지표합산값", i, color=True)

                # 속도값 피봇
                #speed_pivot_df = pivot_table(speed_df, "V_mean", preprocess=modify_frame)
                #display("speed_pivot_df : ", speed_pivot_df)
                #save_to_excel(speed_pivot_df, folder_path, "평균속도", i)

                # 지표별 구성값(속도변화값)
                #speed_pivot_df = pivot_table(speed_df, "delta_V", preprocess=modify_frame)
                #display("speed_pivot_df : ", speed_pivot_df)
                #save_to_excel(speed_pivot_df, folder_path, "속도변화값", i)

                # 지표별 구성값(밀도변화값)
                #density_pivot_df = pivot_table(density_df, "delta_K", preprocess=modify_frame)
                #display("density_pivot_df : ", density_pivot_df)
                #save_to_excel(density_pivot_df, folder_path, "밀도변화값", i)

                # 지표별 구성값(중차량혼입률)
                #heavy_pivot_df = pivot_table(heavy_df, "rate", preprocess=modify_frame)
                #display("heavy_pivot_df : ", heavy_pivot_df)
                #save_to_excel(heavy_pivot_df, folder_path, "중차량혼입률", i)

                # 지표별 구성값(동적포화도)
                #entry_saturation_pivot_df = pivot_table(entry_saturation_df, "Phi_진입", preprocess=modify_frame)
                #display("entry_saturation_pivot_df : ", entry_saturation_pivot_df)
                #save_to_excel(entry_saturation_pivot_df, folder_path, "동적포화도", i)

                # 지표별 구성값(램프 간섭 영향률)
                #rfr_pivot_df = pivot_table(rfr_df, "RFR", preprocess=modify_frame)
                #display("rfr_pivot_df : ", rfr_pivot_df)
                #save_to_excel(rfr_pivot_df, folder_path, "램프간섭영향률", i)

                # 지표별 구성값(진출 원활률)
                #normality_pivot_df = pivot_table(normality_df, "F(outrate)", preprocess=modify_frame)
                #display("normality_pivot_df : ", normality_pivot_df)
                #save_to_excel(normality_pivot_df, folder_path, "진출원활률", i)


                # 메모리 정리
                #del df, original_df, speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df, stvm_df, z_score_df
                gc.collect()


작업파일 :  화성~서울(유입램프 1개)_260127_001.mer


'main_results : '

[      TimeGroup  q_before  q_after     Qm
 0   10200~10500        83      132  107.5
 1   10500~10800        85      123  104.0
 2   10800~11100        98      144  121.0
 3   11100~11400        96      133  114.5
 4   11400~11700        91      136  113.5
 5   11700~12000       102      131  116.5
 6   12000~12300        96      141  118.5
 7   12300~12600       112      150  131.0
 8   12600~12900        92      136  114.0
 9   12900~13200       102      132  117.0
 10    1800~2100       114      149  131.5
 11    2100~2400        88      143  115.5
 12    2400~2700        98      132  115.0
 13    2700~3000       103      141  122.0
 14    3000~3300        98      148  123.0
 15    3300~3600        99      132  115.5
 16    3600~3900       106      144  125.0
 17    3900~4200        97      148  122.5
 18    4200~4500       101      147  124.0
 19    4500~4800       100      144  122.0
 20    4800~5100       118      156  137.0
 21    5100~5400        96      129  112.5
 22    5400

'q_in : '

,TimeGroup,q_in
0,10200~10500,44
1,10500~10800,39
2,10800~11100,42
3,11100~11400,49
4,11400~11700,34
5,11700~12000,33
6,12000~12300,38
7,12300~12600,42
8,12600~12900,40
9,12900~13200,37


...

,TimeGroup,q_before,q_after,Qm,q_in,IR_in,IR_out,New_Measurement,RFR
38,10200~10500,195,251,223.0,63,0.282511,0,317,0.282511
39,10500~10800,181,253,217.0,67,0.308756,0,317,0.308756
40,10800~11100,203,252,227.5,67,0.294505,0,317,0.294505
41,11100~11400,173,264,218.5,64,0.292906,0,317,0.292906
42,11400~11700,204,257,230.5,70,0.303688,0,317,0.303688
43,11700~12000,193,268,230.5,64,0.277657,0,317,0.277657
44,12000~12300,183,247,215.0,69,0.320930,0,317,0.320930
45,12300~12600,186,256,221.0,68,0.307692,0,317,0.307692
46,12600~12900,171,240,205.5,68,0.330900,0,317,0.330900
47,12900~13200,186,254,220.0,68,0.309091,0,317,0.309091


m :  121


'temp : '

,TimeGroup,q_before,q_after,Qm,q_in,IR_in,IR_out,New_Measurement,RFR
0,10200~10500,199,261,230.0,61,0.265217,0,121,0.265217
1,10500~10800,198,258,228.0,61,0.267544,0,121,0.267544
2,10800~11100,204,257,230.5,59,0.255965,0,121,0.255965
3,11100~11400,201,251,226.0,53,0.234513,0,121,0.234513
4,11400~11700,188,254,221.0,60,0.271493,0,121,0.271493
5,11700~12000,204,244,224.0,56,0.250000,0,121,0.250000
6,12000~12300,191,258,224.5,56,0.249443,0,121,0.249443
7,12300~12600,198,252,225.0,51,0.226667,0,121,0.226667
8,12600~12900,201,257,229.0,53,0.231441,0,121,0.231441
9,12900~13200,196,248,222.0,52,0.234234,0,121,0.234234


STVM_3.xlsx 생성 완료
